In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
!nvidia-smi

Tue Dec 22 17:16:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install gast==0.2.2

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=f606a167805f7ff8a1f1ca7319eebade7b7329104dcfc6fe9ad0fd3d4ce2f8d0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
TensorFlow 1.x selected.
1.15.2


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
dataset_file = np.load("/content/gdrive/My Drive/SINTEL_SR/dataset2.npz")

In [ ]:
dataset_x = dataset_file["lr"].astype(np.float32) / 255
dataset_y = dataset_file["hr"].astype(np.float32) / 255

In [ ]:
len(dataset_x)

880

In [ ]:
np.random.seed(42)
dataset_p = np.random.permutation(len(dataset_x))
x_train = dataset_x[dataset_p[:-100]]
y_train = dataset_y[dataset_p[:-100]]
x_valid = dataset_x[dataset_p[-100:-50]]
y_valid = dataset_y[dataset_p[-100:-50]]
x_test = dataset_x[dataset_p[-50:]]
y_test = dataset_y[dataset_p[-50:]]

In [ ]:
#Note: Here it would be best to also flip the channels, since they represent a specific translation!
x_train = np.concatenate([x_train, np.flip(x_train, axis=2)], axis=0)
y_train = np.concatenate([y_train, np.flip(y_train, axis=2)], axis=0)

In [ ]:
dataset_x = None
dataset_y = None

In [ ]:
plt.imshow(x_train[0,:,:,0])

<Figure size 432x288 with 1 Axes>

In [ ]:
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, UpSampling2D, Conv2DTranspose, SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LeakyReLU, ELU, PReLU
from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, ZeroPadding2D, Cropping2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Input, Flatten, Maximum, Add, Lambda, Reshape, Concatenate
from tensorflow.keras.layers import Dropout

from tensorflow.keras.initializers import RandomNormal

from tensorflow.keras.activations import tanh

from tensorflow.keras.regularizers import l2

from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error, mean_absolute_error, binary_crossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

class Subpixel2D(tf.keras.layers.Layer):
    def __init__(self, scale=2, **kwargs):
        super(Subpixel2D, self).__init__(**kwargs)
        self.scale = scale

    def build(self, input_shape):
        super(Subpixel2D, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        return tuple([input_shape[0],
                    (input_shape[1] * self.scale) if (input_shape[1] != None) else None,
                    (input_shape[2] * self.scale) if (input_shape[2] != None) else None,
                    int(input_shape[3] / (self.scale ** 2))])

    def call(self, x):
        return tf.depth_to_space(x, self.scale)

class CReLU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CReLU, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CReLU, self).build(input_shape)

    def call(self, x):
        return tf.nn.crelu(x, axis=-1)

def SRNet(input_depth=8, output_depth=1, upscale_factor=4, highway_depth=8, depth=4, regul=0.000):
    
    initializer='he_normal'
    initializer_last = RandomNormal(mean=0.0, stddev=0.001)

    inputs = Input(shape=[None, None, input_depth])
    inputs_lr = Input(shape=[None, None, output_depth])
    
    x = inputs

    depth_list = []

    for i in range(depth):
        x = Conv2D(highway_depth, (3, 3), padding='same', kernel_regularizer=l2(regul), kernel_initializer=initializer)(x)
        x = CReLU()(x)

        depth_list.append(x)
    
    x = Concatenate(axis=-1)(depth_list)
    x = Conv2D((highway_depth*upscale_factor*upscale_factor), (1, 1), padding='same', kernel_regularizer=l2(regul), kernel_initializer=initializer)(x)
    x = Subpixel2D(upscale_factor)(x)
    x = CReLU()(x)


    depth_list = []

    for i in range(depth):
        x = Conv2D(highway_depth, (3, 3), padding='same', kernel_regularizer=l2(regul), kernel_initializer=initializer)(x)
        x = CReLU()(x)

        depth_list.append(x)
    
    x = Concatenate(axis=-1)(depth_list)
    x = Conv2D(output_depth, (3, 3), padding='same', kernel_regularizer=l2(regul), kernel_initializer=initializer_last, name="conv_ups_resid")(x)

    x_2 = UpSampling2D(size=(upscale_factor, upscale_factor), interpolation='bilinear')(inputs_lr)
    x = Add()([x, x_2])
    
    model = Model([inputs, inputs_lr], x, name='srnet')
    model.summary(line_length=150)
    
    return model


def BIL(input_depth=1, upscale_factor=4):
  
    input_shape = [None, None, input_depth]
    inputs = Input(shape=input_shape)
    
    x = inputs
    x = UpSampling2D(size=(upscale_factor, upscale_factor), interpolation='bilinear')(x)
    outputs = x
    
    bil = Model(inputs, outputs, name='ed_conv')
    
    bil.summary()
    
    return bil

In [ ]:
K.clear_session()
K.reset_uids()

sess = tf.Session()
sess.as_default()

model = SRNet()
model_s = SRNet(input_depth = 1)
model_bil = BIL()

loss=tf.keras.losses.mean_squared_error

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "srnet"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_1 (InputLayer)                             [(None, None, None, 8)]          0                                                                   
______________________________________________________________________________________________________________________________________________________
conv2d (Conv2D)                                  (None, None, None, 8)            584               input_1[0][0]                                     
______________________________

In [ ]:
model.compile(optimizer=Adam(), loss=loss)
model_s.compile(optimizer=Adam(), loss=loss)

In [ ]:
model_log = model.fit(x=[x_train, x_train[...,0:1]], y=y_train, batch_size=16, epochs=60, validation_data=([x_valid, x_valid[...,0:1]], y_valid))

Train on 1560 samples, validate on 50 samples
Epoch 1/60
1560/1560 [==============================] - 87s 56ms/sample - loss: 0.0022 - val_loss: 0.0022
Epoch 2/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0017 - val_loss: 0.0019
Epoch 3/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0016 - val_loss: 0.0018
Epoch 4/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0015 - val_loss: 0.0018
Epoch 5/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0015 - val_loss: 0.0017
Epoch 6/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0014 - val_loss: 0.0016
Epoch 7/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0014 - val_loss: 0.0016
Epoch 8/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0013 - val_loss: 0.0015
Epoch 9/60
1560/1560 [==============================] - 71s 45ms/sample - loss: 0.0013 - val_loss: 0.0015


In [ ]:
model_s_log = model_s.fit(x=[x_train[...,0:1], x_train[...,0:1]], y=y_train, batch_size=16, epochs=60, validation_data=([x_valid[...,0:1], x_valid[...,0:1]], y_valid))

Train on 1560 samples, validate on 50 samples
Epoch 1/60
1560/1560 [==============================] - 80s 51ms/sample - loss: 0.0026 - val_loss: 0.0023
Epoch 2/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0019 - val_loss: 0.0022
Epoch 3/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0018 - val_loss: 0.0020
Epoch 4/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0017 - val_loss: 0.0019
Epoch 5/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0017 - val_loss: 0.0019
Epoch 6/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0017 - val_loss: 0.0019
Epoch 7/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0016 - val_loss: 0.0019
Epoch 8/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0016 - val_loss: 0.0018
Epoch 9/60
1560/1560 [==============================] - 79s 51ms/sample - loss: 0.0016 - val_loss: 0.0018


In [ ]:
model.save_weights('/content/gdrive/My Drive/SINTEL_SR/model_multi-s.hdf5')
model_s.save_weights('/content/gdrive/My Drive/SINTEL_SR/model_single-s.hdf5')

In [ ]:
model.load_weights('/content/gdrive/My Drive/SINTEL_SR/model_multi-s.hdf5')
model_s.load_weights('/content/gdrive/My Drive/SINTEL_SR/model_single-s.hdf5')

In [ ]:
import PIL

In [ ]:
for midx in range(50):
    bpredy = model_bil.predict(x_test[midx:midx+1,:,:,0:1])
    PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save("results/"+str(midx)+"_LR.png")
    bpredy = model_s.predict([x_test[midx:midx+1,:,:,0:1], x_test[midx:midx+1,:,:,0:1]])
    PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save("results/"+str(midx)+"_SR_s.png")
    bpredy = model.predict([x_test[midx:midx+1], x_test[midx:midx+1,:,:,0:1]])
    PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save("results/"+str(midx)+"_SR_m.png")
    PIL.Image.fromarray(np.clip(y_test[midx,:,:,0]*255, 0, 255).astype(np.uint8)).save("results/"+str(midx)+"_HR.png")

In [ ]:
!zip -r results.zip results/

In [ ]:
midx = 0
figsize = (20, 20)
plt.figure(figsize=figsize)
plt.imshow(np.clip(x_test[midx,:,:,0]*255, 0, 255).astype(np.uint8), cmap='gray', vmin=0, vmax=255)
PIL.Image.fromarray(np.clip(x_test[midx,:,:,0]*255, 0, 255).astype(np.uint8)).save(str(midx)+"_LR.png")
plt.show()

In [ ]:
bpredy = model_s.predict([x_test[midx:midx+1,:,:,0:1], x_test[midx:midx+1,:,:,0:1]])
plt.figure(figsize=figsize)
plt.imshow(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8), cmap='gray', vmin=0, vmax=255)
PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save(str(midx)+"_SR_s.png")
plt.show()

In [ ]:
bpredy = model.predict([x_test[midx:midx+1], x_test[midx:midx+1,:,:,0:1]])
plt.figure(figsize=figsize)
plt.imshow(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8), cmap='gray', vmin=0, vmax=255)
PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save(str(midx)+"_SR_m.png")
plt.show()

In [ ]:
bpredy = model.predict([np.concatenate([x_test[midx:midx+1,:,:,:5],x_test[midx:midx+1,:,:,5:]*0], axis=-1), x_test[midx:midx+1,:,:,0:1]])
plt.figure(figsize=figsize)
plt.imshow(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8), cmap='gray', vmin=0, vmax=255)
PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save(str(midx)+"_SR_m_d5.png")
plt.show()

In [ ]:
for i in range(9):
    bpredy = model.predict([np.concatenate([x_test[midx:midx+1,:,:,:i],x_test[midx:midx+1,:,:,i:]*0], axis=-1), x_test[midx:midx+1,:,:,0:1]])
    PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save(str(midx)+"_SR_m_d"+str(i)+".png")

In [ ]:
bpredy = model.predict([np.concatenate([x_test[midx:midx+1,:,:,:1],x_test[midx:midx+1,:,:,1:]*0], axis=-1), x_test[midx:midx+1,:,:,0:1]])
plt.figure(figsize=figsize)
plt.imshow(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8), cmap='gray', vmin=0, vmax=255)
PIL.Image.fromarray(np.clip(bpredy[0,:,:,0]*255, 0, 255).astype(np.uint8)).save(str(midx)+"_SR_m_d1.png")
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.imshow(np.clip(y_test[midx,:,:,0]*255, 0, 255).astype(np.uint8), cmap='gray', vmin=0, vmax=255)
PIL.Image.fromarray(np.clip(y_test[midx,:,:,0]*255, 0, 255).astype(np.uint8)).save(str(midx)+"_HR.png")
plt.show()

In [ ]:
print(model_log.history["loss"])
print(model_log.history["val_loss"])

[0.00216628607446089, 0.0017393825730929772, 0.0016085513306256288, 0.0015411925388691134, 0.0014842524276807523, 0.001428673658758784, 0.0013784582828147672, 0.0013291466664761688, 0.001287824967995477, 0.0012576823409360188, 0.0012210762737175592, 0.0011938380558067598, 0.0011681487915129998, 0.0011460833144015992, 0.0011257214053796653, 0.0011112805205182388, 0.0010842041966194907, 0.0010685791768945564, 0.0010476623793156483, 0.0010339895555845056, 0.0010190701031555923, 0.0010026451898738741, 0.0009892069209271517, 0.0009794658954398563, 0.0010037395058390803, 0.0009564036584029404, 0.0009450678952420369, 0.0009394771443345608, 0.0009264639632489819, 0.0009252011107328611, 0.0009106636187658669, 0.0009030870024640209, 0.0008976955268269357, 0.0008875525335614115, 0.0008846956108195278, 0.0008768103601864706, 0.0008748249001562213, 0.0008674687523848544, 0.000865127685933541, 0.0008560982049227907, 0.0008511531483143186, 0.0008499015122652054, 0.0008513066654977126, 0.0008408147946

In [ ]:
print(model_s_log.history["loss"])
print(model_s_log.history["val_loss"])

[0.002635876157989678, 0.0019136794520398744, 0.0017695654028405746, 0.0017234613390591665, 0.0017064526354750762, 0.0016673870145892486, 0.0016429694578385889, 0.0016239386577254686, 0.0016145886447376165, 0.001592527100076087, 0.001571402384732396, 0.0015581433888142689, 0.0015494108516484117, 0.001539773637882601, 0.0015284562728200585, 0.0015244552626823768, 0.0015140512140276723, 0.0015143943395322331, 0.0015024533005765616, 0.0015009599755733058, 0.0014916685248653476, 0.0014866538966695467, 0.0014840390073517576, 0.0014798897399137226, 0.001475966961767811, 0.0014788024067783202, 0.0014680498954243004, 0.0014667536979779984, 0.0014632923044980719, 0.0014598038578883578, 0.0014693650906571212, 0.0014521650134179836, 0.0014502507980721883, 0.0014493966272148566, 0.0014461973962636712, 0.0014461237346180356, 0.0014400689223876748, 0.0014543913510771324, 0.0014360574987502053, 0.0014339274255176766, 0.0014337951299519492, 0.0014383787271351769, 0.0014305738965049385, 0.0014370170642

In [ ]:
def mse(y, y_pred):
    return np.mean((y - y_pred)**2)

def psnr(y, y_pred):
    return 10*np.log10(1/mse(y, y_pred))

In [ ]:
print("M:", mse(model.predict([x_test, x_test[:,:,:,0:1]]), y_test))
print("S:", mse(model_s.predict([x_test[:,:,:,0:1], x_test[:,:,:,0:1]]), y_test))

M: 0.00090441474
S: 0.0016531938


In [ ]:
print("M:", psnr(model.predict([x_test, x_test[:,:,:,0:1]]), y_test))
print("S:", psnr(model_s.predict([x_test[:,:,:,0:1], x_test[:,:,:,0:1]]), y_test))

M: 30.43632366009988
S: 27.81676220198868


In [ ]:
for i in range(9):
    print(i, "M:", psnr(model.predict([np.concatenate([x_test[:,:,:,:i],x_test[:,:,:,i:]*0], axis=-1), x_test[:,:,:,0:1]]), y_test))

0 M: 26.009136344123846
1 M: 27.41327053232613
2 M: 28.051565429955517
3 M: 28.213430699473193
4 M: 28.672523856137925
5 M: 29.211990156471238
6 M: 29.928810040265983
7 M: 30.081713495021454
8 M: 30.43632366009988
